# Clean 2021 Data
In previous processing, some columns were removed to minimize the number of columns in the data.

However, the data set contained more than 3M rows of data.  

Turns out we need to keep the following columns:
- 'DEVICE_REPORT_PRODUCT_CODE'
- 'FOI_TEXT'

In [1]:
# set the year
year = '2021'

In [2]:
# Identify the data directory, working directory, and data files
data_directory = f"./04-Process-{year}-Data"
working_directory = f"./06-Clean-{year}-Data"
data_file = f"{data_directory}/{year}_data_complete.csv"

import os

# Create the working directory if needed
try:
    os.makedirs(working_directory, exist_ok=True)
except OSError as error:
    print(f"Error creating {working_directory}: {error}")



In [3]:
import pandas as pd


# Read the data into a pandas dataframe
data = pd.read_csv(data_file, # The data file being read, from the variable assignment above
                   on_bad_lines='warn', # This tells Pandas to only warn on bad lines vs causing an error
                   dtype = 'str')       # This tells Pandas to treat all numbers as words

In [4]:
# Replace any records that Pandas converted to 'N/A' with an empty string.
data.fillna('', inplace=True)

In [5]:
print(f"Number of: (Rows, Columns) = {data.shape}")

Number of: (Rows, Columns) = (4454884, 34)


## Remove Unwanted Columns

In [6]:
# Remove unwanted columns
unwanted_columns = [
    'MDR_REPORT_KEY',
    'MDR_TEXT_KEY',
    'TEXT_TYPE_CODE',
    'PATIENT_SEQUENCE_NUMBER',
    'DATE_REPORT',
    'DEVICE_SEQUENCE_NO',
    'BRAND_NAME',
    'MANUFACTURER_D_NAME',
    'MODEL_NUMBER',
    'DEVICE_AVAILABILITY',
    'REPORT_NUMBER',
    'REPORT_SOURCE_CODE',
    'NUMBER_DEVICES_IN_EVENT',
    'DATE_RECEIVED',
    'INITIAL_REPORT_TO_FDA',
    'MANUFACTURER_G1_NAME',
    'REMEDIAL_ACTION',
    'EVENT_TYPE',
    'MANUFACTURER_NAME',
    'TYPE_OF_REPORT',
    'SUMMARY_REPORT',
    'NOE_SUMMARIZED',
    #'UDI-DI',
    #'UDI-PUBLIC',
]

data.drop(unwanted_columns, axis=1, inplace=True)

In [7]:
print(f"Number of: (Rows, Columns) = {data.shape}")

Number of: (Rows, Columns) = (4454884, 12)


In [8]:
data

,FOI_TEXT,DEVICE_PROBLEM_CODE,DEVICE_PROBLEM_TEXT,GENERIC_NAME,DEVICE_REPORT_PRODUCT_CODE,UDI-DI,UDI-PUBLIC,DATE_OF_EVENT,REPORTER_OCCUPATION_CODE,REPORT_DATE,EVENT_LOCATION,SOURCE_TYPE
0,(B)(4). MEDICAL DEVICE EXPIRATION DATE: NA. A ...,1535,"Incorrect, Inadequate or Imprecise Resultor Re...",CORONAVIRUS ANTIGEN DETECTION SYSTEM,QKP,00382902560661,00382902560661,11/03/2020,100,,,"00,06"
1,IT WAS REPORTED THAT WHILE USING BD VERITOR PL...,1535,"Incorrect, Inadequate or Imprecise Resultor Re...",CORONAVIRUS ANTIGEN DETECTION SYSTEM,QKP,00382902560661,00382902560661,11/03/2020,100,,,"00,06"
2,THE CUSTOMER REPORTED PROBLEM WAS CONFIRMED. T...,1069,Break,"PUMP, INFUSION",FRN,10885403424267,10885403424267,,003,,I,OTHER
3,THE CUSTOMER REPORTED PROBLEM WAS CONFIRMED. T...,1135,Crack,"PUMP, INFUSION",FRN,10885403424267,10885403424267,,003,,I,OTHER
4,THE CUSTOMER REPORTED PROBLEM WAS CONFIRMED. T...,1153,Degraded,"PUMP, INFUSION",FRN,10885403424267,10885403424267,,003,,I,OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...
4454879,(B)(4). CURRENTLY IT IS UNKNOWN WHETHER OR NOT...,1496,Device Difficult to Program or Calibrate,"ARTIFICIAL PANCREAS DEVICE SYSTEM, THRESHOLD S...",OZO,000000643169736054,(01)000000643169736054,12/27/2021,000,,,04
4454880,INFORMATION RECEIVED BY MEDTRONIC INDICATED TH...,1112,Computer Software Problem,"PUMP, INFUSION, INSULIN, TO BE USED WITH INVAS...",OYC,000000763000317140,(01)000000763000317140,12/07/2021,000,,,"01,07"
4454881,INFORMATION RECEIVED BY MEDTRONIC INDICATED TH...,3010,Power Problem,"PUMP, INFUSION, INSULIN, TO BE USED WITH INVAS...",OYC,000000763000317140,(01)000000763000317140,12/07/2021,000,,,"01,07"
4454882,(B)(4). THE INSULIN PUMP INVOLVED IN THIS EVEN...,1112,Computer Software Problem,"PUMP, INFUSION, INSULIN, TO BE USED WITH INVAS...",OYC,000000763000317140,(01)000000763000317140,12/07/2021,000,,,"01,07"


## Cleaning Data by Dropping Rows Matching Specific Criteria

Use [this answer on Stack Overflow](https://stackoverflow.com/questions/13851535/how-to-delete-rows-from-a-pandas-dataframe-based-on-a-conditional-expression) as a reference for dropping rows from a dataframe using regular expressesions.

In summary:
```
new_data_frame = previous_data_frame.drop(previous_data_frame[CONDITION GOES HERE; ie, previous_data_frame.COLUMN_NAME == "Some Text"].index)
```

### Drop rows where GENERIC_NAME starts with "UNK" ("UNKNOWN", "UNKOWN", or "UNK")

In [9]:
# Drop rows where GENERIC_NAME starts with "UNK" ("UNKNOWN", "UNKOWN", or "UNK")
remove_generic_name_starts_with_unk = data.drop(data[data.GENERIC_NAME.str.contains(r'^UNK')].index)

print(f"Previous row count = {data.shape[0]}")
print(f"New row count      = {remove_generic_name_starts_with_unk.shape[0]}")
print(f"Rows removed       = {data.shape[0] - remove_generic_name_starts_with_unk.shape[0]}")

Previous row count = 4454884
New row count      = 4452143
Rows removed       = 2741


### Drop rows where DEVICE_PROBLEM_TEXT == "Insufficient Information"

In [10]:
# Drop rows where DEVICE_PROBLEM_TEXT == "Insufficient Information"
remove_device_problem_text_insufficient_information = remove_generic_name_starts_with_unk.drop(remove_generic_name_starts_with_unk[remove_generic_name_starts_with_unk.DEVICE_PROBLEM_TEXT == "Insufficient Information"].index)

print(f"Previous row count = {remove_generic_name_starts_with_unk.shape[0]}")
print(f"New row count      = {remove_device_problem_text_insufficient_information.shape[0]}")
print(f"Rows removed       = {remove_generic_name_starts_with_unk.shape[0] - remove_device_problem_text_insufficient_information.shape[0]}")

Previous row count = 4452143
New row count      = 4403329
Rows removed       = 48814


### Drop rows where GENERIC_NAME is a number

In [11]:
# Drop rows where GENERIC_NAME is a number
remove_generic_name_is_number = remove_device_problem_text_insufficient_information.drop(remove_device_problem_text_insufficient_information[remove_device_problem_text_insufficient_information.GENERIC_NAME.str.match(r'^\d+$')].index)
print(f"Previous row count = {remove_device_problem_text_insufficient_information.shape[0]}")
print(f"New row count      = {remove_generic_name_is_number.shape[0]}")
print(f"Rows removed       = {remove_device_problem_text_insufficient_information.shape[0] - remove_generic_name_is_number.shape[0]}")

Previous row count = 4403329
New row count      = 4403292
Rows removed       = 37


### Drop rows where GENERIC_NAME is blank


In [12]:
# Drop rows where GENERIC_NAME is blank
remove_generic_name_is_blank = remove_generic_name_is_number.drop(remove_generic_name_is_number[remove_generic_name_is_number.GENERIC_NAME == ''].index)
print(f"Previous row count = {remove_generic_name_is_number.shape[0]}")
print(f"New row count      = {remove_generic_name_is_blank.shape[0]}")
print(f"Rows removed       = {remove_generic_name_is_number.shape[0] - remove_generic_name_is_blank.shape[0]}")

Previous row count = 4403292
New row count      = 4388346
Rows removed       = 14946


### Drop Rows Where FOI_TEXT == '(B)(4).'
[USING TEXT MINING OF FDA REPORTS TO INFORM EARLY SIGNAL DETECTION OF
CARDIOVASCULAR LEAD RECALLS](https://dashboard.digitalcollections.cuanschutz.edu/downloads/326bf216-7e24-40b3-80b5-2c9afda1da55)

In [13]:
# Drop Rows Where FOI_TEXT == '(B)(4).'
remove_foitext_equals_b4_1 = remove_generic_name_is_blank.drop(remove_generic_name_is_blank[remove_generic_name_is_blank.FOI_TEXT.str.match(r'(^\(B\)\s?\(4\)\s?\.$)+')].index)

print(f"Previous row count = {remove_generic_name_is_blank.shape[0]}")
print(f"New row count      = {remove_foitext_equals_b4_1.shape[0]}")
print(f"Rows removed       = {remove_generic_name_is_blank.shape[0] - remove_foitext_equals_b4_1.shape[0]}")

Previous row count = 4388346
New row count      = 4146093
Rows removed       = 242253


In [14]:
# Drop Rows Where FOI_TEXT == '(B)(4). (B)(4).'
remove_foitext_equals_b4_2 = remove_foitext_equals_b4_1.drop(remove_foitext_equals_b4_1[remove_foitext_equals_b4_1.FOI_TEXT == '(B)(4). (B)(4).'].index)

print(f"Previous row count = {remove_foitext_equals_b4_1.shape[0]}")
print(f"New row count      = {remove_foitext_equals_b4_2.shape[0]}")
print(f"Rows removed       = {remove_foitext_equals_b4_1.shape[0] - remove_foitext_equals_b4_2.shape[0]}")

Previous row count = 4146093
New row count      = 4146061
Rows removed       = 32


## Count the Product Code Occurrences

In [15]:
product_code_occurrences = remove_foitext_equals_b4_2.groupby(['DEVICE_REPORT_PRODUCT_CODE']).size().to_frame('COUNT')
product_code_occurrences.sort_values(by=['COUNT'], ascending=False).head(10)

,COUNT
DEVICE_REPORT_PRODUCT_CODE,
DZE,690942
FRN,529091
QBJ,297367
OZP,203393
QFG,176681
PZE,140709
OYC,136969
OZO,116307
LZG,71943


In [16]:
product_code_occurrences.sort_values(by=['COUNT'], ascending=False).to_csv(f"{working_directory}/product_code_occurrences.csv")

### Identify Rows to Keep Based on Count of Product Code Occurrences
- QBJ

In [17]:
# Drop rows where DEVICE_REPORT_PRODUCT_CODE is not QBJ
remove_device_product_code_not_qbj = remove_foitext_equals_b4_2.drop(remove_foitext_equals_b4_2[remove_foitext_equals_b4_2.DEVICE_REPORT_PRODUCT_CODE != 'QBJ'].index)
print(f"Previous row count = {remove_foitext_equals_b4_2.shape[0]}")
print(f"New row count      = {remove_device_product_code_not_qbj.shape[0]}")
print(f"Rows removed       = {remove_foitext_equals_b4_2.shape[0] - remove_device_product_code_not_qbj.shape[0]}")

Previous row count = 4146061
New row count      = 297367
Rows removed       = 3848694


In [18]:
#remove_device_product_code_not_qbj.to_csv(f"{working_directory}/2021_data_clean.csv")
remove_foitext_equals_b4_2.to_csv(f"{working_directory}/{year}_data_clean.csv")